<a href="https://colab.research.google.com/github/andresmg42/Proyectos-ADAII/blob/main/Dinamic_Solution_Proyect_1_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
class Subject:
  def __init__(self,code,quotas,id):
    self.code:str=code
    self.quotas:int=quotas
    self.index_id:int=id

class Student:
  def __init__(self,code,solicited_subjects,insatisfaction=0):
    self.code:str=code
    self.solicited_subjects:List[SolicitedSubject]=solicited_subjects
    self.assigned_subjects:List[SolicitedSubject]=[]


  def return_info(self):
    string=f'\n{self.code}\n'
    for s_subject in self.solicited_subjects:
       string+=s_subject.return_info()
    return string

  def return_info_sol(self):
    string=f'\n{self.code}\n'
    for s_subject in self.assigned_subjects:
       string+=s_subject.return_info()
    return string


  def calculate_insatisfaction(self):

    if len(self.solicited_subjects)==0:
      return 0.0

    assigned_subject_codes={subject.code for subject in self.assigned_subjects}
    maj_size=len(self.assigned_subjects)
    msj_size=len(self.solicited_subjects)
    if msj_size==0:
      return 0.0
    unsupplied_subjects=[solicited for solicited in self.solicited_subjects if solicited.code not in assigned_subject_codes ]
    sum_pjl=sum(solicited_subject.priority for solicited_subject in unsupplied_subjects)
    fun_Y= msj_size*3-1

    return (1-(maj_size/msj_size))*(sum_pjl/fun_Y)


  def assign_subject(self,subject_code):

    solicited_subject=next((solicited_subject for solicited_subject in self.solicited_subjects if solicited_subject.code==subject_code),None)
    if solicited_subject:
      self.assigned_subjects.append(solicited_subject)


class SolicitedSubject:
  def __init__(self,subject,priority):
    self.subject=subject
    self.priority:int=priority


  def return_info(self):
    return f'({self.subject.code},{self.priority})'

  @property
  def code(self) -> str:
    return self.subject.code

  @property
  def i(self)-> int:
    return self.subject.index_id

In [86]:
from typing import List
#implement an example to test
subjects=[
  Subject('M1',3,0),
  Subject('M2',4,1),
  Subject('M3',2,2)
]

students=[
    Student('e1',[SolicitedSubject(subjects[0],5),
                  SolicitedSubject(subjects[1],2),
                  SolicitedSubject(subjects[2],1)
                  ],1),
    Student('e2',[SolicitedSubject(subjects[0],4),
                  SolicitedSubject(subjects[1],1),
                  SolicitedSubject(subjects[2],3)
                  ],2),
    Student('e3',[SolicitedSubject(subjects[1],3),
                  SolicitedSubject(subjects[2],2)
                  ],3),
    Student('e4',[SolicitedSubject(subjects[0],2),
                  SolicitedSubject(subjects[2],3)
                  ],4),
    Student('e5',[SolicitedSubject(subjects[0],3),
                  SolicitedSubject(subjects[1],2),
                  SolicitedSubject(subjects[2],3)
                  ],5)

]


s=SolicitedSubject(subjects[0],3)
print(s.i)

0


In [87]:
from re import sub
import copy
import math

class DinamicAlogorithm:

  def __init__(self) -> None:
    pass

  def power_set_recursive(self,s):
    result = []

    def backtrack(start, current):
        result.append(current[:])
        for i in range(start, len(s)):
            current.append(s[i])
            backtrack(i + 1, current)
            current.pop()

    backtrack(0, [])
    return result

  def posible_assignments_by_student(self,student):
      solution=[]
      solicited_subjects=student.solicited_subjects
      posible_combinations=self.power_set_recursive(solicited_subjects)
      for combination in posible_combinations:
        new_student=copy.deepcopy(student)
        if len(combination)!=0:
          for solicited_subject in combination:
            new_student.assign_subject(solicited_subject.code)
        solution.append(new_student)

      return solution

  def is_valid(self,student,c):

      assigned_subjects=student.assigned_subjects
      valid=all(c[assigned_subject.i]>=1 for assigned_subject in assigned_subjects)
      return valid

  def c_less_used_quotas_of(self,c,student):

    index_assigned_subjects=[solicited_subject.i for solicited_subject in student.assigned_subjects]

    for i in range(len(c)):
      if i in index_assigned_subjects:
        c[i]-=1
    return c


  def recursive_solve(self,students,subjects):

    c=[subject.quotas for subject in subjects]

    def solve(j,c):

      if j>len(students)-1:
        return 0

      min_local_insatisfaction=math.inf

      posible_assigns_ej=self.posible_assignments_by_student(students[j])

      for student in posible_assigns_ej:

        if self.is_valid(student,c):

          actual_insatisfaction=student.calculate_insatisfaction()

          new_quotas= self.c_less_used_quotas_of(c[:],student)

          future_insatisfaction=solve(j+1,new_quotas)

          total_insatisfaction=actual_insatisfaction + future_insatisfaction

          min_local_insatisfaction=min(min_local_insatisfaction,total_insatisfaction)

      return min_local_insatisfaction

    return solve(0,c)
















In [88]:
al=DinamicAlogorithm()

sol=al.recursive_solve(students,subjects)

print(sol)

# st=al.posible_assignments_by_student(students[0])

# for student in st:
#   print(student.return_info_sol())

# c=al.c_less_used_quotas_of([3,4,2],st[3])

# print(al.is_valid(st[2],[1,1,2]))

# ins=st[3].calculate_insatisfaction()

# print(ins)


0.4916666666666667


In [89]:
from re import sub
import copy
import math

class DinamicMemoAlogorithm:

  def __init__(self) -> None:
    pass

  def power_set_recursive(self,s):
    result = []

    def backtrack(start, current):
        result.append(current[:])
        for i in range(start, len(s)):
            current.append(s[i])
            backtrack(i + 1, current)
            current.pop()

    backtrack(0, [])
    return result

  def posible_assignments_by_student(self,student):
      solution=[]
      solicited_subjects=student.solicited_subjects
      posible_combinations=self.power_set_recursive(solicited_subjects)
      for combination in posible_combinations:
        new_student=copy.deepcopy(student)
        if len(combination)!=0:
          for solicited_subject in combination:
            new_student.assign_subject(solicited_subject.code)
        solution.append(new_student)

      return solution

  def is_valid(self,student,c):

      assigned_subjects=student.assigned_subjects
      valid=all(c[assigned_subject.i]>=1 for assigned_subject in assigned_subjects)
      return valid

  def c_less_used_quotas_of(self,c,student):

    index_assigned_subjects=[solicited_subject.i for solicited_subject in student.assigned_subjects]

    for i in range(len(c)):
      if i in index_assigned_subjects:
        c[i]-=1
    return c


  def dynamic_solve(self,students,subjects):

    memo={}

    c=[subject.quotas for subject in subjects]

    def solve(j,c):

      if j>len(students)-1:
        return 0

      if (j,tuple(c)) in memo:
        return memo[(j,tuple(c))]

      min_local_insatisfaction=math.inf

      posible_assigns_ej=self.posible_assignments_by_student(students[j])

      for student in posible_assigns_ej:

        if self.is_valid(student,c):

          actual_insatisfaction=student.calculate_insatisfaction()

          new_quotas= self.c_less_used_quotas_of(c[:],student)

          future_insatisfaction=solve(j+1,new_quotas)

          total_insatisfaction=actual_insatisfaction + future_insatisfaction

          min_local_insatisfaction=min(min_local_insatisfaction,total_insatisfaction)

      memo[(j,tuple(c))]=min_local_insatisfaction

      return min_local_insatisfaction

    return solve(0,c)

In [90]:
al=DinamicMemoAlogorithm()

sol=al.dynamic_solve(students,subjects)

print(sol)

0.4916666666666667
